In [1]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
from scipy.optimize import newton

st.title("XIRR Calculator with Multiple Tradebooks and Final Value")

# Input for current portfolio value
final_portfolio_value = st.number_input("Enter Current Portfolio Value", min_value=0.0, value=0.0, step=0.01)

# Input for XIRR starting guess
xirr_guess = st.number_input("Enter XIRR Starting Guess (e.g., 0.1 for 10%)", min_value=-1.0, value=0.1, step=0.01)

# File uploader for multiple CSV files
uploaded_files = st.file_uploader("Choose Tradebook CSV files", type="csv", accept_multiple_files=True)

# Define the XIRR functions
def xnpv(rate, values, dates):
    """
    Calculate the Net Present Value of a series of cash flows at irregular intervals.
    """
    if not dates:
        return 0 # Handle empty dates list
    start_date = dates[0]
    return sum(
        cf / (1 + rate) ** ((date - start_date).days / 365.0)
        for cf, date in zip(values, dates)
    )

def xirr(values, dates, guess=0.1, maxiter=10000):
    """
    Calculate the Internal Rate of Return for cash flows at irregular intervals.
    """
    if len(values) < 2: 
        return np.nan

    # Newton's method
    try:
        return newton(
            lambda r: xnpv(r, values, dates), guess, maxiter=maxiter
        )
    except (RuntimeError, ValueError) as e:
        st.error(f"XIRR calculation error: {e}. This can happen if the solver struggles to converge.")
        return np.nan # Return NaN if the solver does not converge or other errors occur


if uploaded_files and final_portfolio_value > 0:
    st.write("Files uploaded and portfolio value entered. Processing...")

    all_trade_data = pd.DataFrame()

    for uploaded_file in uploaded_files:
        try:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(uploaded_file)

            # Drop empty columns
            df = df.dropna(axis=1, how='all')

            # Ensure required columns exist before processing
            required_cols = ['Quantity', 'Price', 'Trade Date', 'Trade Type']
            if not all(col in df.columns for col in required_cols):
                st.warning(f"Skipping file '{uploaded_file.name}': Missing one or more required columns ({', '.join(required_cols)}).")
                continue

            # Remove commas from 'Quantity' and 'Price' columns and convert to numeric
            df['Quantity'] = df['Quantity'].astype(str).str.replace(',', '', regex=False)
            df['Price'] = df['Price'].astype(str).str.replace(',', '', regex=False)
            df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
            df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

            # Drop rows where 'Quantity' or 'Price' are NaN after coercion
            df.dropna(subset=['Quantity', 'Price'], inplace=True)

            # Convert 'Trade Date' to datetime objects
            df['Trade Date'] = pd.to_datetime(df['Trade Date'], format='%d-%m-%Y', errors='coerce')
            df.dropna(subset=['Trade Date'], inplace=True)

            # Convert 'Trade Type' to lowercase for consistent conditional logic
            df['Trade Type'] = df['Trade Type'].astype(str).str.lower()

            # Calculate the cash flow for each transaction
            df['Cash_Flow'] = np.where(df['Trade Type'] == 'buy', -df['Quantity'] * df['Price'], df['Quantity'] * df['Price'])

            all_trade_data = pd.concat([all_trade_data, df], ignore_index=True)

        except Exception as e:
            st.error(f"Error processing file '{uploaded_file.name}': {e}")
            continue

    if not all_trade_data.empty:
        # Group by 'Trade Date' and sum the cash flows to get net cash flow
        cash_flows = all_trade_data.groupby('Trade Date')['Cash_Flow'].sum().reset_index()

        # Sort the cash flows by date
        cash_flows.sort_values(by='Trade Date', inplace=True)

        # Add the final portfolio value as the last cash flow on the latest date
        latest_trade_date = cash_flows['Trade Date'].max()
        final_cash_flow_row = pd.DataFrame([{'Trade Date': latest_trade_date, 'Cash_Flow': final_portfolio_value}])
        cash_flows = pd.concat([cash_flows, final_cash_flow_row], ignore_index=True)

        st.write("Trade data processed and final value added. Calculating XIRR...")

        # Extract dates and cash flows into lists for XIRR calculation
        dates = cash_flows['Trade Date'].tolist()
        cash_flows = cash_flows['Cash_Flow'].tolist()

        # Calculate the XIRR using the user-provided guess
        xirr_value = xirr(values=cash_flows, dates=dates, guess=xirr_guess)

        # Convert the XIRR to a percentage and display
        if not np.isnan(xirr_value):
            xirr_percent = xirr_value * 100
            st.success(f"The calculated XIRR is: {xirr_percent:.2f}%")
        else:
            st.warning("Could not calculate XIRR based on the provided data and portfolio value.")

    elif uploaded_files and all_trade_data.empty:
        st.warning("No valid data found in the uploaded files after processing.")

elif uploaded_files and final_portfolio_value <= 0:
    st.info("Please enter a positive value for the Current Portfolio Value to calculate XIRR.")

elif not uploaded_files and final_portfolio_value > 0:
    st.info("Please upload one or more Tradebook CSV files to calculate XIRR.")

else:
    st.info("Please upload your Tradebook CSV file(s) and enter your Current Portfolio Value to calculate XIRR.")

Overwriting app.py


In [ ]:
!streamlit run app.py